In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df=pd.read_csv('/kaggle/input/air-pollution-in-seoul/AirPollutionSeoul/Measurement_summary.csv')
df.head()

,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5
0,2017-01-01 00:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0
1,2017-01-01 01:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.058,0.002,1.2,71.0,59.0
2,2017-01-01 02:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,59.0
3,2017-01-01 03:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.004,0.056,0.002,1.2,70.0,58.0
4,2017-01-01 04:00,101,"19, Jong-ro 35ga-gil, Jongno-gu, Seoul, Republ...",37.572016,127.005008,0.003,0.051,0.002,1.2,69.0,61.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647511 entries, 0 to 647510
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Measurement date  647511 non-null  object 
 1   Station code      647511 non-null  int64  
 2   Address           647511 non-null  object 
 3   Latitude          647511 non-null  float64
 4   Longitude         647511 non-null  float64
 5   SO2               647511 non-null  float64
 6   NO2               647511 non-null  float64
 7   O3                647511 non-null  float64
 8   CO                647511 non-null  float64
 9   PM10              647511 non-null  float64
 10  PM2.5             647511 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 54.3+ MB


In [4]:
# Let's look at the total number of the variable 'Station code.'
df['Station code'].unique()

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125])

In [5]:
set(df['Station code'])

{101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125}

# Select and prepare data

In [6]:
list_stations = [111, 112, 113, 114, 115, 116, 117, 118]
df_select = df[df['Station code'].isin(list_stations)]
df_select.head()

,Measurement date,Station code,Address,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5
259000,2017-01-01 00:00,111,"70, Samyang-ro 2-gil, Seongbuk-gu, Seoul, Repu...",37.606719,127.027279,0.005,0.047,0.002,0.9,62.0,38.0
259001,2017-01-01 01:00,111,"70, Samyang-ro 2-gil, Seongbuk-gu, Seoul, Repu...",37.606719,127.027279,0.005,0.046,0.002,0.9,61.0,38.0
259002,2017-01-01 02:00,111,"70, Samyang-ro 2-gil, Seongbuk-gu, Seoul, Repu...",37.606719,127.027279,0.004,0.043,0.002,0.9,57.0,43.0
259003,2017-01-01 03:00,111,"70, Samyang-ro 2-gil, Seongbuk-gu, Seoul, Repu...",37.606719,127.027279,0.004,0.042,0.002,0.9,54.0,45.0
259004,2017-01-01 04:00,111,"70, Samyang-ro 2-gil, Seongbuk-gu, Seoul, Repu...",37.606719,127.027279,0.004,0.044,0.002,1.0,48.0,43.0


The retrieved dataset is not ready to be plotted. Some columns are needed to be created or modified before use.